This can be used to initalize the database for many of the different experiments below. 

In [11]:
from Juliet_DB_loader import TestCase, Folder, SubFolder, sessionmaker, engine
from sqlalchemy import func
from sqlalchemy.orm import sessionmaker 
import re

Session = sessionmaker(bind=engine)
session = Session()


The below code can be used to verify a specific test cases's content and line number before any modifications are made


In [ ]:
test_cases = session.query(TestCase).filter_by(id=47105).first()
line_content = dict()
lines = test_cases.file_content.split('\n')
multiline_comment = False
for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line

# Print the dictionary vertically
for line_num, line in line_content.items():
    print(f"{line_num}: {line}")

This block can be used to check the vulnerable line of a specific test file. 

In [34]:
test_case = session.query(TestCase).filter_by(id=10000).first()
line_content = dict()
lines = test_case.file_content.split('\n')
vulnerable_line = int(test_case.vulnerability_location)

print(f"Vulnerable line: {vulnerable_line}")
print(f"Total lines in file: {len(lines)}")

for line_num, line in enumerate(lines, start=1):
    line_content[line_num] = line    

# Check if vulnerable_line is a valid key in line_content
if vulnerable_line in line_content:
    print(f"{vulnerable_line}: {line_content[vulnerable_line]}")
else:
    print(f"Vulnerable line {vulnerable_line} is not in the file content.")

Vulnerable line: 43
Total lines in file: 133
43:         strcpy(data, source);


2-6-24 


This block's goal is to get a batch of files and put them into a new folder for experimenting my comment_cleaner.

2-7 
I have sucessfully done this, and I am going to take a batch of programs from each langauge to make my first attempt at a corpus for tokenization. 

Corpus Version 1.0
Important Notes: I am using the suboptimal comment stripper so there are a fair amount of comments still in this corpus. 


In [18]:
from Juliet_DB_loader import TestCase, Folder, SubFolder, sessionmaker, engine
from sqlalchemy import and_, func
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.exc import SQLAlchemyError
from comment_cleaner import strip
from comment_cleaner import remove_empty_lines
import random
Session = sessionmaker(bind=engine)
session = Session()

# To DO get a selection of 10 of test cases of each language
# Run them through the comment cleaner
# Manually check the results

# Function to get 10 random test cases for a given file extension
def get_test_cases(session, file_extension, num_cases):
    query = session.query(TestCase)

    test_cases = query.filter(TestCase.file_extension == file_extension).limit(num_cases).all()

    # Return the list of test cases and the corresponding IDs as a manifest
    test_case_ids = [test_case.id for test_case in test_cases]
    return test_cases, test_case_ids


c_test_cases, c_ids = get_test_cases(session, '.c', 3000)
cpp_test_cases, cpp_ids = get_test_cases(session, '.cpp', 3000)
java_test_cases, java_ids = get_test_cases(session, '.java', 3000)

all_cases = c_test_cases + cpp_test_cases + java_test_cases
manifest = c_ids + cpp_ids + java_ids
corpus_file_path = 'corpus_1.0.txt'
manifest_file_path = 'manifest_1.0.txt'

with open(manifest_file_path, 'w', encoding='utf-8') as manifest_file:
    # Write each original test case ID to the manifest file
    for case_id in manifest:
        manifest_file.write(f"{case_id}\n")

for case in all_cases:
    lines = case.file_content.split('\n')
    stripped_lines, _, _ = strip(lines)
    stripped_lines = remove_empty_lines(stripped_lines)
    case.file_content = '\n'.join(stripped_lines)
    #print(f"Test Case {case.id}:")
    #for line_number, line_content in enumerate(stripped_lines, start=1):    
    #     print(f" {line_content}")
    # print("-" * 50)
    with open(corpus_file_path, 'a', encoding='utf-8') as corpus_file:
        corpus_file.write('\n'.join(stripped_lines) + '\n')
